In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/examples-of-calls'):
    for filename in filenames:
        pass
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/examples-of-calls/samples/implantcity.mp3
/kaggle/input/examples-of-calls/samples/Звонок в техподдержку КорбинаТелеком - Во дает!!)).mp3
/kaggle/input/examples-of-calls/samples/Звонок в техподдержку - Пьяная мадам.mp3
/kaggle/input/examples-of-calls/samples/Пранкер - Звонок в техподдержку антивируса Лаборатория Касперского....mp3
/kaggle/input/examples-of-calls/samples/whiteservice.mp3
/kaggle/input/examples-of-calls/samples/Звонок в техподдержку Stream - Антон Уральский (Ни единого разрыва!).mp3
/kaggle/input/examples-of-calls/samples/ЗВОНОК КЛИЕНТА СТРИМА В ТЕХПОДДЕРЖКУ - ВЫ НЕ ОТВЕЧАЕТЕ НА МОЙ ОТВЕТ!!!.mp3
/kaggle/input/examples-of-calls/samples/Nightmare - Звонок в техподдержку Rambler.mp3
/kaggle/input/examples-of-calls/samples/79180970824_in_2022_06_04-12_58_30_79184924202_3pqb.mp3
/kaggle/input/examples-of-calls/samples/Звонок в техподдержку - Матрица xxx твои мозги,прикинь.mp3


In [68]:
from glob import glob
from pathlib import Path
from tqdm.notebook import tqdm

__import__('warnings').filterwarnings("ignore")

In [69]:
from pydub import AudioSegment
from transformers import HubertForSequenceClassification, Wav2Vec2FeatureExtractor
import torchaudio
import torch

# Указать устройство (GPU) для вычислений, если оно доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели и экстрактора признаков на устройство GPU
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertForSequenceClassification.from_pretrained("xbgoose/hubert-speech-emotion-recognition-russian-dusha-finetuned")
model.to(device)

HubertForSequenceClassification(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertLayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): HubertFeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (pro

In [70]:
DATASET_PATH = r'/kaggle/input/sample-wavs'
DATASETS = ['podcast_train', 'podcast_test', 'crowd_test']
EXAMPLES = Path(r'/kaggle/input/examples-of-calls')
WORK_PATH = Path('.')

num2emotion = {0: 'neutral', 1: 'angry', 2: 'positive', 3: 'sad', 4: 'other'}
emotion2num = {emotion: num for num, emotion in num2emotion.items()}

dataset = DATASETS[-1]

crowd_test = os.path.join(DATASET_PATH, dataset)
crowd_info = os.path.join(crowd_test, f'raw_{dataset}.tsv')

In [71]:
df = pd.read_csv(crowd_info, sep='\t')
print('Всего строк:', len(df))

Всего строк: 79088


In [72]:
check_column = 'annotator_emo'
# check_column = 'speaker_emo'

if check_column != 'speaker_emo':
    df["speaker_emo"].fillna(df["annotator_emo"], inplace=True)
else:
    df.dropna(subset=['speaker_emo'], inplace=True)

df.drop_duplicates(subset=['audio_path', check_column], inplace=True)
print('Без дубликатов:', len(df))

if not len(df):
    print('\nФАЙЛЫ ДЛЯ ОБРАБОТКИ ОТСУТСТВУЮТ !!!')
    exit()

tqdm.pandas()
# Проверка и фильтрация датафрейма
df = df[df['audio_path'].progress_apply(lambda x: os.path.isfile(os.path.join(crowd_test, x)))]
print('Итого осталось файлов:', len(df))

Без дубликатов: 24616


  0%|          | 0/24616 [00:00<?, ?it/s]

Итого осталось файлов: 24616


In [73]:
emos = pd.DataFrame(df.groupby('audio_path')[check_column].apply(lambda x: list(x)))
emos['audio_path'] = emos.index
emos.reset_index(drop=True, inplace=True)
emos.columns = ['ann_emos', 'audio_path']
df_flt = df.merge(emos, on='audio_path', how='left')
df_flt['ann_emos'] = df_flt['ann_emos'].map(lambda x: ' '.join(sorted(x)))

# убираем записи в который эмоция "angry" встречается с другими эмоциями
df_flt = df_flt[~df_flt['ann_emos'].str.startswith('angry ')]
# убираем записи в который эмоция "positive" встречается с другими эмоциями
df_flt = df_flt[~df_flt['ann_emos'].str.contains(' positive')]  # После фильтрации: 9991
df_flt = df_flt[~df_flt['ann_emos'].str.contains('positive ')]  # После фильтрации: 9987
print('После фильтрации:', len(df_flt))

df = df_flt

После фильтрации: 17986


In [74]:
%%time
tmp = df.iloc[:]
preds = []
predicted_probs = []
for index, row in tqdm(tmp.iterrows(), total=len(tmp)):
    file_path = os.path.join(crowd_test, row['audio_path'])
    
    waveform, sample_rate = torchaudio.load(file_path, normalize=True)
    transform = torchaudio.transforms.Resample(sample_rate, 16000)
    waveform = transform(waveform)
    
    # Перенести данные на GPU
    waveform = waveform.to(device)

    inputs = feature_extractor(
            waveform, 
            sampling_rate=feature_extractor.sampling_rate, 
            return_tensors="pt",
            padding=True,
            max_length=16000 * 10,
            truncation=True
        )

    # Перенести данные на GPU
    inputs = {key: val.to(device) for key, val in inputs.items()}

    logits = model(inputs['input_values'][0]).logits
    predictions = torch.argmax(logits, dim=-1)
    # Перенести результаты на CPU
    pred = predictions.cpu().numpy()[0]
    predicted_emotion = num2emotion[pred]

    preds.append(predicted_emotion)
#     print('pred:', predicted_emotion, 'true:', row['annotator_emo'])

    # Преобразование меток классов в вероятности (softmax)
    softmax = torch.nn.Softmax(dim=1)
    probabilities = softmax(logits)
    predicted_probs.append(probabilities.detach().cpu().numpy())
    
tmp['pred'] = preds
tmp['probs'] = predicted_probs

tmp.to_csv(f'preds_hb_{dataset}.csv', index=False)

  0%|          | 0/17986 [00:00<?, ?it/s]

CPU times: user 15min 6s, sys: 5.99 s, total: 15min 12s
Wall time: 17min 37s


In [75]:
from IPython.display import display, FileLink
from zipfile import ZipFile, ZIP_DEFLATED as ZD
from glob import glob

files = glob(f'{WORK_PATH}/*.csv')
zip_filename = WORK_PATH.joinpath('predictions.zip')
with ZipFile(zip_filename, 'w',  compression=ZD, compresslevel=9) as zip_file:
    for filename in files:
        print(filename)
        zip_file.write(filename)
FileLink(zip_filename)

./preds_hb_crowd_test.csv
./preds_hb_podcast_test.csv


/kaggle/working/predictions.zip

In [77]:
phb = tmp

In [82]:
from sklearn.metrics import f1_score as F1, classification_report, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

tst = phb.copy()

remap = {'sad': 'neutral', 'other': 'neutral'}

for col in ('annotator_emo', 'speaker_emo', 'pred'):
    tst[col] = tst[col].map(lambda x: remap.get(x, x))

emos = {'5 emo': phb, '3 emo': tst}
true_columns = ('annotator_emo', 'speaker_emo')

for emo, df in emos.items():
    for name_col in ('annotator_emo', 'speaker_emo'):
        # Пример истинных меток и предсказанных вероятностей для многоклассовой классификации
        # Истинные метки классов
        y_true = df[name_col].map(emotion2num)  
        # Предсказанные вероятности для каждого класса
        y_scores = np.concatenate(df['probs'].values)  
        # Преобразуем истинные метки классов в бинарный формат
        lb = LabelBinarizer()
        y_true_bin = lb.fit_transform(y_true)
        n_emos = len(lb.classes_)
        roc_auc = roc_auc_score(y_true_bin, y_scores[:, :n_emos], average="macro") 
        
        print(f'f1_score ({emo}) по колонке {name_col:<13} =',
              F1(df[name_col], df['pred'], average='weighted', zero_division=0).round(3),
             'roc_auc_score = ', roc_auc)

f1_score (5 emo) по колонке annotator_emo = 0.718 roc_auc_score =  0.8761457372133957
f1_score (5 emo) по колонке speaker_emo   = 0.731 roc_auc_score =  0.905502759992634
f1_score (3 emo) по колонке annotator_emo = 0.966 roc_auc_score =  0.9470148750107813
f1_score (3 emo) по колонке speaker_emo   = 0.9 roc_auc_score =  0.8751866786978845


In [84]:
for emo, df in emos.items():
    for name_col in ('annotator_emo', 'speaker_emo'):
        print(f'{emo} по колонке {name_col:<13}:')
        print(classification_report(df[name_col], df['pred'], zero_division=0))

5 emo по колонке annotator_emo:
              precision    recall  f1-score   support

       angry       0.85      0.88      0.86      1278
     neutral       0.74      0.83      0.78     10372
       other       0.89      0.31      0.46       377
    positive       0.83      0.85      0.84      1015
         sad       0.60      0.48      0.54      4944

    accuracy                           0.73     17986
   macro avg       0.78      0.67      0.70     17986
weighted avg       0.72      0.73      0.72     17986

5 emo по колонке speaker_emo  :
              precision    recall  f1-score   support

       angry       0.92      0.60      0.73      2019
     neutral       0.67      0.93      0.78      8320
       other       0.00      0.00      0.00         0
    positive       0.88      0.57      0.69      1594
         sad       0.85      0.56      0.68      6053

    accuracy                           0.74     17986
   macro avg       0.66      0.53      0.57     17986
weighted avg 